In [316]:
import pickle
import csv
from scipy.spatial.distance import cosine
from Palate import Palate
import json
pal = Palate()

### Create SQLlite3

In [397]:
import sqlite3
from sqlite3 import Error
 
 
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn
 
 
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def create_row_DURHAM(conn, food_info):
 
    sql = f''' INSERT INTO DURHAM(item_name, latitude, longitude, restaurant_name, item_description, price, embedding)
              VALUES(?,?,?,?,?,?,?) '''

    cur = conn.cursor()
    cur.execute(sql, food_info)
    return cur.lastrowid

def create_user(conn, user_id, password):
 
    sql = f''' INSERT INTO USERS(user_id, password, joined_date)
              VALUES(?,?,?) '''

    joined_date = 'todays'
    cur = conn.cursor()
    info = (user_id, password, joined_date)
    cur.execute(sql, info)
    return cur.lastrowid

def add_preference(conn, user_id, accepted):
 
    sql = f''' INSERT INTO PREFS(user_id, preferences)
              VALUES(?,?) '''

    cur = conn.cursor()
    info = (user_id, accepted)
    cur.execute(sql, info)
    return cur.lastrowid

In [309]:
# !rm 'nutrics.db'

In [325]:
#Create database + connection
conn = create_connection('nutrics.db')
cursor = conn.cursor()

In [326]:
#create durham food table with params from csv file below
durham_food_table = "CREATE TABLE DURHAM (item_name NOT NULL, latitude NOT NULL, longitude NOT NULL, restaurant_name NOT NULL, item_description NOT NULL, price NOT NULL, embedding NOT NULL)"
create_table(conn,durham_food_table)

table DURHAM already exists


In [327]:
#populate food database with contents of csv file. WILL FAIL if there are missing inputs (NOT NULL parameter)
#csv file uses a | delimiter because some food entries use ','

with open('/Users/joshuadarcy/Documents/nutrics/food.csv') as csv_file: 
    csv_reader = csv.reader(csv_file, delimiter='|')
    line_count = 0
    for row in csv_reader: 
        if line_count == 0:
            print(f'Column names are: {", ".join(row)}')
            line_count += 1
        else: 
            item = row[0]
            palate = pal.palate_constructor(item)
            jsoned = json.dumps(palate.tolist()) #SQL doesn't accept arrays so need to serialize into JSON strings
            create_row_DURHAM(conn, row + [jsoned])

Column names are: item_name, latitude, longitude, restaurant_name, item_description, price


In [330]:
#database testing

#find {limit} items closest to {cur_lat, cur_lng}
limit = 400
cur_lat = 35.880705
cur_lng = -78.849392
e = cursor.execute(f'''
SELECT rowid, embedding FROM DURHAM ORDER BY (({cur_lat}-latitude)*({cur_lat}-latitude)) + (({cur_lng} - longitude)*({cur_lng} - longitude)) ASC LIMIT {limit};
''')
embedded = e.fetchall()

#parse and reconstruct into a list of the embeddings within a given area
item_names = []
vectors = []
for vector in embedded:
    item_names.append(vector[0])
    loaded = json.loads(vector[1])
    vectors.append(loaded)
  

numped = np.array(vectors)

In [332]:
pref = pal.palate_constructor('steak')

matches = [cosine(pref, local_item) for local_item in numped]
best_match = min(matches)
index = matches.index(best_match)
e = cursor.execute(f'''
SELECT item_name, latitude, longitude, restaurant_name, item_description, price FROM DURHAM WHERE rowid = {item_names[index]};
''')
e.fetchall()

[('Roast Beef Gyro',
  '35.87831',
  '-78.85005',
  "Arby's",
  'Nothing says “I am an adventurous eater and interesting person” like eating a gyro at Arby’s. Our classic thinly sliced roast beef is topped with lettuce, onions and tomatoes, cool creamy tzatziki sauce, and authentic Greek seasonings all hugged by a warm pita.',
  '')]

In [78]:
cursor.execute('SELECT longitude FROM DURHAM')

In [370]:
cursor.execute('DROP TABLE USERS')

In [371]:
#create users table
users_table = "CREATE TABLE USERS (user_id NOT NULL, password NOT NULL, joined_date NOT NULL)"
create_table(conn,users_table)

In [372]:
create_user(conn, 'joshuadrc', 'pass123')
create_user(conn, 'shaina', 'pass123')

2

In [374]:
e = cursor.execute('SELECT * FROM USERS')
e.fetchall()

[('joshuadrc', 'pass123', 'todays'), ('shaina', 'pass123', 'todays')]

In [410]:
def login(username, password): 
    e = cursor.execute(f'SELECT password FROM USERS WHERE user_id = "{user}";').fetchone()[0]
    if e == password: 
        return True
    else: 
        return False

def get_preferences(username): 
    

In [420]:
authenticated = login('joshuadrc', 'pass123')
if authenticated: 
    add_preference(conn, 'joshuadrc', 'steak')

#check history of preferences
    e = cursor.execute(f'SELECT preferences FROM PREFS WHERE user_id = "joshuadrc";').fetchall()
    print(e)

[('cheeseburger with fries',), ('cheeseburger with fries',), ('cheeseburger with fries',), ('steak and beans',), ('steak and beans',), ('steak',)]


In [408]:
#Handle login

user = "joshuadrc"
password = 'pass1234'

e = cursor.execute(f'SELECT password FROM USERS WHERE user_id = "{user}";').fetchone()[0]
if e == password:
    print(True)
else:
    print(False)

False


In [393]:
#create preferences table
pref_table = "CREATE TABLE PREFS (user_id NOT NULL, preferences NOT NULL)"
create_table(conn, pref_table)

In [404]:
add_preference(conn, 'shaina', 'kale salad')

4

In [406]:
e = cursor.execute(f'SELECT preferences FROM PREFS WHERE user_id = "shaina";').fetchall()
e

[('kale salad',)]